In [3]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
# Importando o módulo de Regressão Linear do scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

In [2]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código de Neg."),on="Código de Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
#Removando uma coluna de notas que nao foi utilizada, pois tem dados faltando
uniaoAbas.drop("NotaFundamentei20200201",axis=1,inplace=True)

In [5]:
#Selecionando todas as empresas que possuem notas e estão com seus dados completos
empresasComNotas=uniaoAbas.replace("-",np.nan)
empresasComNotas.dropna(inplace=True)

In [18]:
empresasComNotas['Avaliação'] = empresasComNotas['NotaFundamentei'] >= 3
empresasComNotas.replace(True,"Boa",inplace=True)
empresasComNotas.replace(False,"Ruim",inplace=True)

In [20]:
dfCNota=normalizaDf(empresasComNotas)

In [22]:
#Gerando a analise de correlação dos atributos, já excluindo a "NotaFundamentei" da resposta
analise=analiseCorr("NotaFundamentei",dfCNota)[1:]

In [25]:
#Selecionando os melhores atributos para treinar o regressor
melhoresAtributos=[]
count=0
for k in analise.values:
    if (k > 0.3) | (k< -0.3):
        melhoresAtributos.append(analise.axes[0][count])

In [24]:
X,y=dfCNota[melhoresAtributos], empresasComNotas["Avaliação"]

In [60]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
y_test

149     Boa
277     Boa
136     Boa
63     Ruim
228     Boa
178     Boa
356     Boa
322    Ruim
263    Ruim
175     Boa
Name: Avaliação, dtype: object

In [29]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [30]:
gnb.fit(X_train, y_train)

GaussianNB()

In [31]:
Y_prev=gnb.predict(X_test)

In [32]:
Y_prev

array(['Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Ruim',
       'Boa'], dtype='<U4')

In [35]:
#Criando um outro dataFrame para vizualizar NotaFundamentei x NotaPredita 
emp=empresasComNotas.loc[X_test.index.values,["Nome","Código de Neg.","NotaFundamentei","Avaliação"]]

In [36]:
emp["AvaliaçãoPredita"]=Y_prev

In [37]:
emp.sort_values("NotaFundamentei",ascending=False)

,Nome,Código de Neg.,NotaFundamentei,Avaliação,AvaliaçãoPredita
136,Engie Brasil,EGIE3,4.8,Boa,Boa
228,Lojas Renner,LREN3,4.8,Boa,Boa
149,Fleury,FLRY3,4.6,Boa,Boa
175,Hypera,HYPE3,4.3,Boa,Boa
277,Qualicorp,QUAL3,3.6,Boa,Boa
178,Iguatemi,IGTA3,3.3,Boa,Boa
356,Totvs,TOTS3,3.3,Boa,Boa
322,Smiles,SMLS3,2.7,Ruim,Boa
263,Petrobras,PETR3,2.7,Ruim,Ruim
63,BR Malls Par,BRML3,2.6,Ruim,Boa


In [61]:
from sklearn.metrics import accuracy_score,classification_report

# Avaliar a precisão
print("Acurácia: ",accuracy_score(y_test, Y_prev))

0.8


In [64]:
print(classification_report(y_test, Y_prev))

Clasification report
              precision    recall  f1-score   support

         Boa       0.78      1.00      0.88         7
        Ruim       1.00      0.33      0.50         3

    accuracy                           0.80        10
   macro avg       0.89      0.67      0.69        10
weighted avg       0.84      0.80      0.76        10



In [73]:
def get_confusion_matrix(reais, preditos, labels):
    """
    Uma função que retorna a matriz de confusão para uma classificação binária
    
    Args:
        reais (list): lista de valores reais
        preditos (list): lista de valores preditos pelo modelos
        labels (list): lista de labels a serem avaliados.
            É importante que ela esteja presente, pois usaremos ela para entender
            quem é a classe positiva e quem é a classe negativa
    
    Returns:
        Um numpy.array, no formato:
            numpy.array([
                [ tp, fp ],
                [ fn, tn ]
            ])
    """
    # não implementado
    if len(labels) > 2:
        return None

    if len(reais) != len(preditos):
        return None
    
    # considerando a primeira classe como a positiva, e a segunda a negativa
    true_class = labels[0]
    negative_class = labels[1]

    # valores preditos corretamente
    tp = 0
    tn = 0
    
    # valores preditos incorretamente
    fp = 0
    fn = 0
    
    for (indice, v_real) in enumerate(reais):
        v_predito = preditos[indice]

        # se trata de um valor real da classe positiva
        if v_real == true_class:
            tp += 1 if v_predito == v_real else 0
            fp += 1 if v_predito != v_real else 0
        else:
            tn += 1 if v_predito == v_real else 0
            fn += 1 if v_predito != v_real else 0
    
    return np.array([
        # valores da classe positiva
        [ tp, fp ],
        # valores da classe negativa
        [ fn, tn ]
    ])

matrizDeConfusao=get_confusion_matrix(reais=y_test, preditos=Y_prev, labels=["Boa","Ruim"])
# array([[3, 1], [2, 4]])

In [74]:
matrizDeConfusao
'''
    numpy.array([
                [ tp, fp ],
                [ fn, tn ]
            ])
'''

array([[7, 0],
       [2, 1]])